This notebook helps with plotting and interpretation of the global seismic wavefield generated by the AxiSEM3D software (https://github.com/AxiSEMunity/AxiSEM3D) and processed as .sac files as in write_3D_record_section.ipynb.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
from matplotlib.colors import ListedColormap
import scipy as sp
from scipy import signal, fft
from scipy.signal import decimate
from subprocess import run
import re
import pandas as pd
import obspy
import obspy.signal.freqattributes as freqatt
from obspy.core.util.attribdict import AttribDict

import plotly
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

In [ ]:
### establish paths and variables ###
# location of AxiSEM3D master directory
axisem_loc = "~/Documents/Synthetics/axisem3d_root/AxiSEM3D"

# this notebook assumes that the simulation input, output directories etc are located in 
# run_dir = f'{axisem_loc}/{model_name}/simu{run_dims}/{run_name}'
# adjust file paths as necessary for your setup

model_name = 'LayeredCore_4Hz'
run_name = '2DVQC400'
run_dims = '3D'

run_dir = f'{axisem_loc}/{model_name}/simu{run_dims}/{run_name}'

# master directory where processed seismic traces will be stored
working_dir = "~/Documents/Enceladus/3D"

# choose which station file (affects path to processed data)
station_dims = '1D'
station_dict = {'1D': 'stations.txt',
               '3D': '3Dstations.txt'}
station_file = station_dict[station_dims]

# set location of processed seismic data
output_dir = f'{working_dir}/{model_name}/simu{station_dims}/{run_name}'

# create directory to store output figures
run(['mkdir', '-p', f'{output_dir}/figures'])

# which component to plot
component = "Z"

# datatype names compatible with write_record_section notebook
datatype = "filt_1_6" # e.g. filt_1_6, raw

# set write = 1 to save output figure
write = 0

In [ ]:
sta_lon = {}
sta_lat = {}

# read list of stations
with open(f'{run_dir}/input/{station_file}','r') as file:
    for line in file:
        try:
            p = line.split()
            float(p[0])
            sta_lat[str(p[0])]=p[2]
            sta_lon[str(p[0])]=p[3]
        except:
            pass

if station_dims == '1D':
    staspacing = abs(float(sta_lat["0001"]) - float(sta_lat["0002"]))
else:
    staspacing = abs(float(sta_lat["0101"]) - float(sta_lat["0102"]))

In [ ]:
# set every nth sample to plot (reduces jupyter notebook memory requirements to speed up plotting)
downsample = 5

# adjust the scale of the plot (will autoscale the amplitude bar)
scale_factor = (2e5)

# choose constants for plotting lat and lon (e.g. fix_stalon = '13' selects stations at 0 deg E)
fix_stalat = '19'
fix_stalon = '00'

# set range of x-axis (in seconds)
xrange = [0,400]

# choose what type of plot to make (see Dapré & Irving 2025 for examples)
plot_type = 'record' # record or core_belt or belt

# set figure properties etc. depending on plot type
if plot_type == 'record':
    st = obspy.read(f'{output_dir}/{fix_stalon}*{component}{datatype}.sac',headonly=True)
    fig=go.Figure(layout=go.Layout(title=go.layout.Title(text=f'{model_name}_{run_name}_{component}_{datatype}_{plot_type}_{fix_stalon}',x=0.01,y=0.9)))
    fig_save = f"{output_dir}/figures/{model_name}_{run_name}_{component}_{datatype}_{plot_type}_{fix_stalon}.png"
    staspacing_func = lambda i_station: staspacing*(i_station)-90
    yrange = [-90,90]
    ytitle = "Latitude (deg)"
    fig_width = 1000
    fig_height = 600
    amp_loc_y = 60
    amp_loc_x = 200
    scale_adjust = 1
elif plot_type == 'belt':
    st = obspy.read(f'{output_dir}/*{fix_stalat}{component}{datatype}.sac',headonly=True)
    fig=go.Figure(layout=go.Layout(title=go.layout.Title(text=f'{run_name}_{component}_{datatype}_{plot_type}_{fix_stalat}',x=0.01,y=0.9)))
    fig_save = f"{output_dir}/figures/{model_name}_{run_name}_{component}_{datatype}_{plot_type}_{fix_stalat}.png"
    staspacing_func = lambda i_station: 2*staspacing*(i_station)-180
    yrange = [-194,179]
    ytitle = "Longitude (deg)"
    amp_loc_y = -180
    amp_loc_x = 20
    scale_adjust = 2
    fig_width = 500
    fig_height = 600
    xrange=[0,300]
elif plot_type == 'core_belt':
    st = obspy.read(f'{output_dir}/*{fix_stalat}{component}{datatype}.sac',headonly=True)
    fig=go.Figure(layout=go.Layout(title=go.layout.Title(text=f'{run_name}_{component}_{datatype}_{plot_type}_{fix_stalat}',x=0.01,y=0.9)))
    fig_save = f"{output_dir}/figures/{model_name}_{run_name}_{component}_{datatype}_{plot_type}_{fix_stalat}.png"
    staspacing_func = lambda i_station: 2*staspacing*(i_station)-180
    yrange = [-194,179]
    xrange = [170,310]
    ytitle = "Longitude (deg)"
    fig_width = 500
    fig_height = 600
    amp_loc_y = -150
    amp_loc_x = 188
    scale_adjust = 2
i = 0

# import time array and downsample
time = np.loadtxt(f'{run_dir}/output/stations/Enceladus_stations/data_time.ascii')
time = decimate(time,downsample)

for tr in st:
    i = i+1
    if i>2 and i<25: # limited to avoid plotting source/antipode which are usually unphysically high-amplitude
        s2plot = tr.stats['station']    
        data = obspy.read(f'{output_dir}/{s2plot}{component}{datatype}.sac', headonly=False)[0]
        data.decimate(downsample)
        plotfiltdata = [float(x)*scale_factor + staspacing_func(i-1) for x in data]
        # example functionality to highlight one trace (change line color)
        if i == 10:
            fig.add_trace(go.Scattergl(x=np.array(time), y=np.array(plotfiltdata),name=f"{s2plot}",line=dict(color='blue',width=0.6))) 
        else:
            fig.add_trace(go.Scatter(x=np.array(time), y=np.array(plotfiltdata),name=f"{s2plot}",line=dict(color='blue',width=0.6))) 
        
# create generic function to plot vertical lines such as for amplitude scaling
line = np.linspace(-1*2, 2, 20)
length = len(line)
func = lambda length, arrival: [arrival]*length

# produce amplitude scalebar with automatic magnitude calculation
plotline = [scale_adjust*float(x)*2 + float(amp_loc_y) for x in line]
plotend = [float(y)*0.7 + float(amp_loc_x) for y in line]

fig.add_trace(go.Scatter(x=func(length,amp_loc_x),y=plotline,line=dict(color='crimson',width=0.8)))
fig.add_trace(go.Scatter(x=plotend,y=func(length,amp_loc_y-scale_adjust*4),line=dict(color='crimson',width=0.7)))
fig.add_trace(go.Scatter(x=plotend,y=func(length,amp_loc_y+scale_adjust*4),line=dict(color='crimson',width=0.7)))

# 2*2 comes from the height of line being +/- 2 which is then multiplied by 2 in plotline
scale_amplitude = int(2*2*scale_adjust/scale_factor * 10**6)

# choose amplitude label depending on what units are being plotted
fig.add_trace(go.Scatter(x=[amp_loc_x+2],y=[amp_loc_y-3],mode="text",text = [f"Amplitude: \u00B1 {int(scale_amplitude)} \u03BCm/s\N{SUPERSCRIPT TWO}"], textposition="bottom right",textfont=dict(color='crimson')))
#fig.add_trace(go.Scatter(x=[amp_loc_x+2],y=[amp_loc_y-3],mode="text",text = [f"Amplitude: \u00B1 {int(scale_amplitude)} nm/s"], textposition="bottom right",textfont=dict(color='crimson')))

fig.update_layout(width=fig_width, height = fig_height, showlegend=False, xaxis_title="Time (s)", yaxis_title=ytitle, plot_bgcolor='rgba(0,0,0,0)')
fig.update_xaxes(rangemode='tozero',range=xrange,ticks="inside", showline=True, linecolor='#444',title_font_color='#444')
fig.update_yaxes(range=yrange,ticks="inside", dtick=15, showline=True, linecolor='#444',title_font_color='#444')
fig.show()

if write == 1:
    fig.write_image(fig_save,scale=4)